In [1]:
import sys
sys.path.append('../scripts/')
from utility import read_file, create_folder, temp_record_query, temp_record_sdf

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = (
    # Create a spark session (which will run spark jobs)
    SparkSession.builder.appName("Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config('spark.executor.memory','10g')
    .config('spark.driver.memory','12g')
    .config('spark.driver.maxResultsSize', '10 GiB')
    .config('spark.shuffle.file.buffer', '64k')
    # .config("spark.network.timeout", "3600s")
    # .master("local[6]")
    .getOrCreate()
    )

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/14 12:30:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/14 12:30:30 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### 1. Connect Merchants and Transactions AS `df`

In [4]:
print("================ Merchant ================")
merchants_sdf = read_file(spark, 'split_tags.parquet', '../data/merchants_preprocess/', truncate=20)

================ Merchant ================
|> Loading File...


|> Loading Finished!
-RECORD 0-----------------------------
 merchant_name | Felis Limited        
 tags          | furniture, home f... 
 merchant_abn  | 10023283211          
 take_rate     | 0.18                 
 type          | e                    
only showing top 1 row



In [5]:
print("================ Transactions ================")
transactions_sdf = read_file(spark, 'transactions_20210228_20210827_snapshot')

================ Transactions ================
|> Loading File...


|> Loading Finished!
-RECORD 0----------------------------------------------
 user_id        | 18478                                
 merchant_abn   | 62191208634                          
 dollar_value   | 63.255848959735246                   
 order_id       | 949a63c8-29f7-4ab0-ada4-99ac50a88952 
 order_datetime | 2021-08-20                           
only showing top 1 row



In [6]:
df = transactions_sdf.join(merchants_sdf, how='left', on='merchant_abn')

### 2. `df` join with Customer

In [7]:
customer_sdf = read_file(spark, 'consumer_data.parquet', '../data/curated/')

|> Loading File...
|> Loading Finished!
-RECORD 0---------------------------------
 name        | Courtney Mendez            
 address     | 802 Knapp Harbors Apt. 769 
 state       | QLD                        
 postcode    | 9013                       
 gender      | Female                     
 consumer_id | 680810                     
only showing top 1 row



In [8]:
df.show(3)

+------------+-------+------------------+--------------------+--------------+--------------------+--------------------+---------+----+
|merchant_abn|user_id|      dollar_value|            order_id|order_datetime|       merchant_name|                tags|take_rate|type|
+------------+-------+------------------+--------------------+--------------+--------------------+--------------------+---------+----+
| 62191208634|  18478|63.255848959735246|949a63c8-29f7-4ab...|    2021-08-20|Cursus Non Egesta...|furniture, home f...|     2.17|   c|
| 15549624934|      2| 130.3505283105634|6a84c3cf-612a-457...|    2021-08-20|  Commodo Associates|opticians, optica...|     2.76|   c|
| 64403598239|  18479|120.15860593212783|b10dcc33-e53f-425...|    2021-08-20|Lobortis Ultrices...|music shops - mus...|     6.31|   a|
+------------+-------+------------------+--------------------+--------------+--------------------+--------------------+---------+----+
only showing top 3 rows



In [9]:
df = df.join(customer_sdf, df.user_id == customer_sdf.consumer_id, 'left')

### 3. `df` join with the external dataset 

In [10]:
rrm_sdf = read_file(spark, 'external_rrm.parquet', '../data/curated/', truncate=20)

|> Loading File...
|> Loading Finished!


-RECORD 0----------
 Date | 2020-02-21 
 AUS  | 1          
 NSW  | 1          
 VIC  | 1          
 QLD  | -1         
 SA   | 2          
 WA   | 2          
 TAS  | -1         
 NT   | 4          
 ACT  | 4          
only showing top 1 row



In [11]:
df.show(5)

+------------+-------+------------------+--------------------+--------------+--------------------+--------------------+---------+----+--------------+---------------+-----+--------+------+-----------+
|merchant_abn|user_id|      dollar_value|            order_id|order_datetime|       merchant_name|                tags|take_rate|type|          name|        address|state|postcode|gender|consumer_id|
+------------+-------+------------------+--------------------+--------------+--------------------+--------------------+---------+----+--------------+---------------+-----+--------+------+-----------+
| 96442520325|      7|29.863817960377617|48dd50dd-291d-43d...|    2021-04-07|Nullam Lobortis Q...|computers, comput...|     3.65|   b|James Williams|3709 Mary River|  TAS|    7248|  Male|          7|
| 29566626791|      7| 80.43460573195489|13d12435-4f1d-4b6...|    2021-04-07|                null|                null|     null|null|James Williams|3709 Mary River|  TAS|    7248|  Male|          7|


In [12]:
rrm_sdf.show(5)

+----------+---+---+---+---+---+---+---+---+---+
|      Date|AUS|NSW|VIC|QLD| SA| WA|TAS| NT|ACT|
+----------+---+---+---+---+---+---+---+---+---+
|2020-02-21|  1|  1|  1| -1|  2|  2| -1|  4|  4|
|2020-02-22|  1|  1|  1| -1|  2|  1| -1|  5|  6|
|2020-02-23|  1|  0|  1| -1|  2|  2| -1|  6|  7|
|2020-02-24|  1|  1|  1| -1|  2|  2| -1|  6|  7|
|2020-02-25|  1|  1|  2| -1|  3|  1| -1|  7|  8|
+----------+---+---+---+---+---+---+---+---+---+
only showing top 5 rows



In [21]:
df = df.join(rrm_sdf, df.order_datetime == rrm_sdf.Date, 'left')

In [28]:
cols = ['state', 'AUS', 'NSW', 'VIC', 'QLD', 'SA', 'WA', 'TAS', 'NT', 'ACT']
df.select(cols).toDF('state', 'AUS', 'NSW', 'VIC', 'QLD', 'SA', 'WA', 'TAS', 'NT', 'ACT')

state,AUS,NSW,VIC,QLD,SA,WA,TAS,NT,ACT
null,-24,-47,-39,1,2,11,-6,11,-59
null,-24,-47,-39,1,2,11,-6,11,-59
null,-24,-47,-39,1,2,11,-6,11,-59
null,-24,-47,-39,1,2,11,-6,11,-59
null,-24,-47,-39,1,2,11,-6,11,-59
null,-24,-47,-39,1,2,11,-6,11,-59
null,-24,-47,-39,1,2,11,-6,11,-59
null,-24,-47,-39,1,2,11,-6,11,-59
NSW,-24,-47,-39,1,2,11,-6,11,-59
NSW,-24,-47,-39,1,2,11,-6,11,-59
